In [ ]:
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical


def tfrecord2array(path_res):
    imgs = []
    lbls = []
    # print('tfrecords_files to be transformed:', path_res)
    reader = tf.TFRecordReader()

    filename_queue = tf.train.string_input_producer([path_res])

    # 从 TFRecord 读取内容并保存到 serialized_example 中
    _, serialized_example = reader.read(filename_queue)
    # 读取 serialized_example 的格式
    features = tf.parse_single_example(
        serialized_example,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
        })

    # 解析从 serialized_example 读取到的内容
    labels = tf.cast(features['label'], tf.int64)
    images = tf.decode_raw(features['image_raw'], tf.uint8)

    # print('Extracting {} has just started.'.format(path_res))
    with tf.Session() as sess:
        # 启动多线程
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        # try:
        flag_stop = 0
        while not coord.should_stop():
            if flag_stop == 2:
                # 连续两张和刚开始读入时相等, 则认为读取结束, 弹出末尾重复的两张
                imgs.pop()
                imgs.pop()
                lbls.pop()
                lbls.pop()
                break
            label, img = sess.run([labels, images])
            if not len(imgs):
                img_first = img.copy()
            elif len(imgs) == 1:
                img_second = img.copy()
            else:
                if (not sum(img.ravel() != img_first.ravel()) or
                        not sum(img.ravel() != img_second.ravel())):
                    flag_stop += 1
                else:
                    flag_stop = 0

            # plt.imshow(img, plt.cm.gray)
            # plt.title(str(label))
            # plt.show()
            img = (img > 0).astype(np.uint8).reshape(-1)
            imgs.append(img)
            lbls.append(label)
            if not len(lbls) % 100:
                print("lbls.shape:", np.array(lbls).shape)

        # except tf.errors.OutOfRangeError as err:
        #     print('Extracting {} has been finished.'.format(
        #                                             path_res))
        # finally:
        coord.request_stop()
        coord.join(threads)
    return to_categorical(np.array(lbls), num_classes=70), np.array(imgs)


def main():
    imgs, labels = tfrecord2array(
        r"./data_tfrecords/integers_tfrecords/test.tfrecords")
    print("imgs.shape:", imgs.shape)
    print("labels.shape:", labels.shape)

In [ ]:
if __name__ == '__main__':
    main()